In [10]:
d = '../data/pre/' # raw data directory

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [11]:
aC = pd.read_csv('../data/pre/app_categories.csv')

user_installedapps = pd.read_csv('%suser_installedapps.csv' % d)
ui_cates = pd.merge(user_installedapps, aC, on='appID', how='left')
ui_dum_cats = pd.get_dummies(ui_cates['appCategory'], prefix='appCategory', dummy_na=True)
ui_cates = pd.concat([ui_cates, ui_dum_cats], axis=1)
print('\n\n',ui_cates.head(), len(ui_cates))



    userID  appID  appCategory  appCategory_0.0  appCategory_1.0  \
0       1    357          201                0                0   
1       1    360          201                0                0   
2       1    362          409                0                0   
3       1    365          301                0                0   
4       1    375          203                0                0   

   appCategory_2.0  appCategory_103.0  appCategory_104.0  appCategory_105.0  \
0                0                  0                  0                  0   
1                0                  0                  0                  0   
2                0                  0                  0                  0   
3                0                  0                  0                  0   
4                0                  0                  0                  0   

   appCategory_106.0       ...         appCategory_401.0  appCategory_402.0  \
0                  0       ...          

In [43]:
ui = ui_cates.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'appCategory_' in col]].values, axis=0))).reset_index()

ui.columns = ['userID', 'insAppCates']

print('\nDev info\n',ui.head(), len(ui))


Dev info
    userID                                        insAppCates
0       1  [37, 0, 0, 1, 0, 0, 0, 2, 0, 0, 9, 5, 0, 1, 0,...
1       9  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2      10  [46, 0, 1, 1, 0, 0, 1, 0, 0, 0, 5, 2, 0, 1, 0,...
3      12  [21, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,...
4      14  [21, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 1, 0,... 1446105


In [44]:
ui.insAppCates.head().values

array([ [37, 0, 0, 1, 0, 0, 0, 2, 0, 0, 9, 5, 0, 1, 0, 1, 8, 0, 0, 1, 0, 0, 1, 3, 0, 2, 8, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [46, 0, 1, 1, 0, 0, 1, 0, 0, 0, 5, 2, 0, 1, 0, 0, 7, 0, 0, 3, 3, 4, 0, 4, 1, 0, 18, 0],
       [21, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 1, 2, 0, 0, 4, 1, 1, 0, 0],
       [21, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 1, 0, 0, 5, 0, 3, 0, 1, 0, 0, 2, 0, 2, 19, 0]], dtype=object)

In [45]:
user_app_actions = pd.read_csv('%suser_app_actions.csv' % d)

uact_cate = pd.merge(user_app_actions, aC, on='appID', how='left')
uact_dum_cats = pd.get_dummies(uact_cate['appCategory'], prefix='appActCategory', dummy_na=True)
uact_cate = pd.concat([uact_cate, uact_dum_cats], axis=1)

print('\n\nDev info...', ...)



Dev info... Ellipsis


In [46]:
uact_cate['installTime_d'] = uact_cate.installTime.map(lambda x: int(str(x)[0:2]))
uact_cate = uact_cate.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'appActCategory_' in col]].values, axis=0))).reset_index()
uact_cate.columns = ['userID', 'actApps']

In [47]:
print('\n\n',uact_cate.head(), len(uact_cate))



    userID                                            actApps
0       1  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       5  [2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, ...
2      12  [2, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...
3      14  [4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      19  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ... 781112


In [48]:
te_df = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)
te_df = pd.merge(te_df, ui, on='userID', how='left')
te_df = pd.merge(te_df, uact_cate, on='userID', how='left')
te_df.to_csv('../data/pre/new_with_lists_test.csv', index=False)

In [61]:
tr_df = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
tr_df = pd.merge(tr_df, ui, on='userID', how='left')
tr_df = pd.merge(tr_df, uact_cate, on='userID', how='left')
tr_df[['actApps', 'insAppCates']].fillna('nan', inplace=True)
tr_df.to_csv('../data/pre/new_with_lists_train.csv', index=False)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [62]:
te_df[['actApps', 'insAppCates']].fillna('nan', inplace=True)
te_df.to_csv('../data/pre/new_with_lists_test.csv', index=False)
len(te_df.actApps.head().values[3])

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


28

In [66]:
te_df_lsts = pd.read_csv('../data/pre/new_with_lists_train.csv')

In [67]:
te_df_lsts.dtypes

label                int64
creativeID           int64
userID               int64
positionID           int64
connectionType       int64
telecomsOperator     int64
adID                 int64
camgaignID           int64
advertiserID         int64
appID                int64
appPlatform          int64
age                  int64
gender               int64
education            int64
marriageStatus       int64
haveBaby             int64
hometown             int64
residence            int64
sitesetID            int64
positionType         int64
appCategory          int64
clickTime_d          int64
weekDay              int64
clickTime_h          int64
clickTime_m          int64
conversionTime_d     int64
insAppCates         object
actApps             object
dtype: object

In [68]:
np.array(eval(te_df_lsts.insAppCates.values[0]))

array([15,  0,  1,  1,  0,  0,  0,  1,  0,  0,  4,  4,  0,  0,  1,  0,  5,
        0,  0,  2,  0,  0,  0,  2,  1,  2,  5,  0])

In [69]:
exit()